In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo-1106",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationBufferMemory(
    llm=llm,
    memory_key="chat_history",
)

memory.save_context({"input":"Hi"}, {"output":"How are you??"})

memory.load_memory_variables({})


{'chat_history': 'Human: Hi\nAI: How are you??'}

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)


memory = ConversationBufferMemory(
    llm=llm,
    memory_key="chat_history",
)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)

output = chain.run("Who is Julia?")

memory.save_context({"input":"Who is Julia?"}, {"output":output})

memory.load_memory_variables({})

c:\Users\gpwns\Git\FULLSTACK-GPT\env\Lib\site-packages\langchain\chains\llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'chat_history': 'Human: Who is Julia?\nAI: Julia is a character in the text that Winston is thinking of transferring his punishment to in order to avoid the rats. She is the person he cares about the most in the world and is willing to endure any torture in her place.'}